# Set up the Environment

In [1]:
using PyCall
py"""
import os
print(os.getcwd())
import sys
sys.path.append(os.getcwd())
"""
py"""
import simple_with_injection
"""
include("../src/SpikingNeuralNetworks.jl")
include("../src/units.jl")
include("../src/plot.jl")
try
    using UnicodePlots
    using PyCall
catch
    using Pkg
    Pkg.add("PyCall")
    Pkg.add("UnicodePlots")
    Pkg.add("Conda")
    using Conda
    Conda.add("matplotlib")
    Pkg.build("PyCall")
    
    using PyCall
    using UnicodePlots
end
SNN = SpikingNeuralNetworks.SNN

py"""
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
matplotlib.get_backend()
"""
py"""
from neuronunit import tests
from neuronunit.tests import fi
from simple_with_injection import SimpleModel
from neo import AnalogSignal
import quantities as pq
try:
   from julia import Main
except:
   import julia
   julia.install()
   from julia import Main


"""

In [2]:

simple = py"SimpleModel()"

py"""
import pickle
from neuronunit.tests.fi import RheobaseTestP

cell_tests = pickle.load(open('multicellular_constraints.p','rb'))
for test in cell_tests.values():
    if "Rheobase test" in test.keys():
        temp_test = {k:v for k,v in test.items()}
        break
rt = temp_test["Rheobase test"]

rtest = RheobaseTestP(observation=rt.observation)
#                        name='RheobaseTest')
JHH = {
'Vr': -68.9346,
'Cm': 0.0002,
'gl': 1.0 * 1e-5,
'El': -65.0,
'EK': -90.0,
'ENa': 50.0,
'gNa': 0.02,
'gK': 0.006,
'Vt': -63.0
}
JHH1 = {k:(v-0.01*v) for k,v in JHH.items()}
JHH2 = {k:(v+0.01*v) for k,v in JHH.items()}

ranges = {k:(v-0.01*v,v+0.01*v) for k,v in JHH.items()}



"""

current_params = py"rt.params"
print(current_params)
simple.attrs = py"JHH"

vm = simple.inject_square_current(current_params)

py"""
model = SimpleModel()
model2 = SimpleModel()
model3 = SimpleModel()
model.attrs = JHH
model2.attrs = JHH1
model3.attrs = JHH2

import pickle
rheo = rt.generate_prediction(model)
print(rheo)
rheo1 = rt.generate_prediction(model2)
print(rheo1)
rheo2 = rt.generate_prediction(model3)
print(rheo2)

"""

py"""


def evaluate(tests):
    from sciunit.scores.collections import ScoreArray
    from sciunit import TestSuite
    import sciunit
    #from neuronunit.optimisation.data_transport_container import DataTC
    scores_ = []
    from collections import OrderedDict
    tests = OrderedDict({k:v for k,v in tests.items()})
    for key, temp_test in tests.items():
        if key == 'Rheobase test':
            temp_test.score_type = sciunit.scores.ZScore
        if key in ['Injected current AP width test', 'Injected current AP threshold test', 'Injected current AP amplitude test', 'Rheobase test']:
            try:
                temp_test.params['injected_square_current']['amplitude'] = rheo['value']
            except:
                pass
        try:
            score = temp_test.judge(model)
        except:
            score = -np.inf
        if isinstance(score, sciunit.scores.incomplete.InsufficientDataScore):
            score = -np.inf
        else:
            score = score
        scores_.append(score)
    tt = TestSuite(list(tests.values()))
    SA = ScoreArray(tt, scores_)
    return SA
cell_tests = pickle.load(open('multicellular_constraints.p','rb'))
scores = []
for tests in cell_tests.values():
    SA = evaluate(tests)
    print(SA)
    scores.append(SA)

"""
sc = py"scores"
@show(sc)
#println()


Dict{Any,Any}("duration"=>1000.0,"tmax"=>1300.0,"padding"=>200.0,"delay"=>100.0,"dt"=>0.025,"amplitude"=>100.0,"tolerance"=>1.0)N = 1
E2.I = 

                        +------------------------------------------------------------+ 
     49.303558349609375 |    |@@@@@@@@@@@@@]@@@@]@@@[@@@@[@@@@@@@@@@@@@@@@@[         | 
                        |    |@@@@@@@@@@@@@]@@@@@@@@[@@@@[@@@@@@@@@@@@@@@@@[         | 
                        |    |@@@@@@@@@@@@@]@@@@@@@@[@@@@[@@@@@@@@@@@@@@@@@[         | 
                        |    |@@@@@@@@@@@@@]@@@@@@@@[@@@@[@@@@@@@@@@@@@@@@@[         | 
                        |    |@@@@@@@@@@@@@]@@@@@@@@[@@@@@@@@@@@@@@@@@@@@@@[         | 
                        |    |@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@[         | 
                        |    |@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@[         | 
                        |----|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@}---------| 
                        |    |@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@[         | 
   v                    |    |@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@[         | 
                        |    |@@

Float32[0.001]


                        +------------------------------------------------------------+ 
       49.5881233215332 |    |@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@         | 
                        |    |@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@         | 
                        |    |@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@         | 
                        |    |@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@         | 
                        |    |@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@         | 
                        |    |@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@         | 
                        |    |@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@         | 
                        |----|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@---------| 
                        |    |@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@         | 
   v                    |    |@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@         | 
                        |    |@@

N = 1
E2.I = Float32[0.003]


...

                        +------------------------------------------------------------+ 
     48.954917907714844 |                                                   |        | 
                        |                                                   |        | 
                        |                                                   |        | 
                        |                                                   |        | 
                        |                                                   |        | 
                        |                                                   |        | 
                        |                                                   |        | 
                        |"""""""""""""""""""""""""""""""""""""""""""""""""""T""""""""| 
                        |                                                   |        | 
   v                    |                                                   |        | 
                        |       

N = 1
E2.I = Float32[1.83147e-5]


                        +------------------------------------------------------------+ 
     -64.77056884765625 |                                                     .r"""""| 
                        |    |                                                /      | 
                        |.  |]                                               ]`      | 
                        ||  /|                                               |       | 
                        ||  ||                                               |       | 
                        || .||                                              ,`       | 
                        || | |                                              |        | 
                        || / |.                                             ]        | 
                        || |  |                                             |        | 
   v                    || |  |                                             |        | 
                        |\ |  | 

N = 1
E2.I = Float32[-0.0001]
sc = PyObject[PyObject Suite_394942361710
InjectedCurrentAPWidthTest        Z = -0.71
InjectedCurrentAPThresholdTest     Z = 0.98
RestingPotentialTest              Z = -0.56
InputResistanceTest               Z = -0.24
InjectedCurrentAPAmplitudeTest     Z = 1.81
TimeConstantTest                       -inf
RheobaseTest                      Z = -1.49
CapacitanceTest                   Z = -2.05
dtype: object, PyObject Suite_196833971188
InjectedCurrentAPWidthTest        Z = -1.74
InjectedCurrentAPThresholdTest     Z = 0.33
RestingPotentialTest               Z = 0.53
InputResistanceTest                Z = 0.07
InjectedCurrentAPAmplitudeTest     Z = 0.67
TimeConstantTest                  Z = -0.69
RheobaseTest                      Z = -1.24
CapacitanceTest                   Z = -0.47
dtype: object]


2-element Array{PyObject,1}:
 PyObject Suite_394942361710
InjectedCurrentAPWidthTest        Z = -0.71
InjectedCurrentAPThresholdTest     Z = 0.98
RestingPotentialTest              Z = -0.56
InputResistanceTest               Z = -0.24
InjectedCurrentAPAmplitudeTest     Z = 1.81
TimeConstantTest                       -inf
RheobaseTest                      Z = -1.49
CapacitanceTest                   Z = -2.05
dtype: object
 PyObject Suite_196833971188
InjectedCurrentAPWidthTest        Z = -1.74
InjectedCurrentAPThresholdTest     Z = 0.33
RestingPotentialTest               Z = 0.53
InputResistanceTest                Z = 0.07
InjectedCurrentAPAmplitudeTest     Z = 0.67
TimeConstantTest                  Z = -0.69
RheobaseTest                      Z = -1.24
CapacitanceTest                   Z = -0.47
dtype: object

In [3]:
@show(sc)


sc = PyObject[PyObject Suite_394942361710
InjectedCurrentAPWidthTest        Z = -0.71
InjectedCurrentAPThresholdTest     Z = 0.98
RestingPotentialTest              Z = -0.56
InputResistanceTest               Z = -0.24
InjectedCurrentAPAmplitudeTest     Z = 1.81
TimeConstantTest                       -inf
RheobaseTest                      Z = -1.49
CapacitanceTest                   Z = -2.05
dtype: object, PyObject Suite_196833971188
InjectedCurrentAPWidthTest        Z = -1.74
InjectedCurrentAPThresholdTest     Z = 0.33
RestingPotentialTest               Z = 0.53
InputResistanceTest                Z = 0.07
InjectedCurrentAPAmplitudeTest     Z = 0.67
TimeConstantTest                  Z = -0.69
RheobaseTest                      Z = -1.24
CapacitanceTest                   Z = -0.47
dtype: object]


2-element Array{PyObject,1}:
 PyObject Suite_394942361710
InjectedCurrentAPWidthTest        Z = -0.71
InjectedCurrentAPThresholdTest     Z = 0.98
RestingPotentialTest              Z = -0.56
InputResistanceTest               Z = -0.24
InjectedCurrentAPAmplitudeTest     Z = 1.81
TimeConstantTest                       -inf
RheobaseTest                      Z = -1.49
CapacitanceTest                   Z = -2.05
dtype: object
 PyObject Suite_196833971188
InjectedCurrentAPWidthTest        Z = -1.74
InjectedCurrentAPThresholdTest     Z = 0.33
RestingPotentialTest               Z = 0.53
InputResistanceTest                Z = 0.07
InjectedCurrentAPAmplitudeTest     Z = 0.67
TimeConstantTest                  Z = -0.69
RheobaseTest                      Z = -1.24
CapacitanceTest                   Z = -0.47
dtype: object